In [1]:
import os
import wikipedia
import json
from langchain_chroma import Chroma
from langchain.docstore.document import Document
from langchain_huggingface import HuggingFaceEmbeddings
import uuid  # Import the uuid module for generating unique IDs

In [2]:
class BiographyVectorStore:
    def __init__(self, scraped_data_dir, vector_store_dir):
        self.scraped_data_dir = scraped_data_dir
        self.vector_store_dir = vector_store_dir

        # Ensure directories exist
        os.makedirs(self.scraped_data_dir, exist_ok=True)
        os.makedirs(self.vector_store_dir, exist_ok=True)

        # Initialize embeddings
        self.model_name = "sentence-transformers/all-mpnet-base-v2"
        self.model_kwargs = {'device': 'cpu'}
        self.encode_kwargs = {'normalize_embeddings': False}
        self.embeddings = HuggingFaceEmbeddings(
            model_name=self.model_name,
            model_kwargs=self.model_kwargs,
            encode_kwargs=self.encode_kwargs
        )

    def get_biography(self, name):
        """Fetches the biography of a person from Wikipedia."""
        try:
            content = wikipedia.page(name).content
            print(f"Successfully fetched biography for {name}.")
            # Optionally, print the first 100 characters of the biography
            print(f"Snippet: {content[:100]}...\n")
            return content
        except Exception as e:
            print(f"Error fetching page for {name}: {e}")
            return ""

    def save_biography(self, name, content):
        """Saves the biography content to a file."""
        file_path = os.path.join(self.scraped_data_dir, f"{name.replace(' ', '_')}.txt")
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(content)

    def load_biographies(self, names):
        """Loads biographies from Wikipedia and saves them locally."""
        texts = []
        for name in names:
            bio = self.get_biography(name)
            if bio:
                self.save_biography(name, bio)
                texts.append((name, bio))
        return texts

    def create_document_store(self, texts):
        """Creates a document store from the texts and persists it."""
        documents = []
        for name, text in texts:
            # Generate a unique ID for each person
            person_id = str(uuid.uuid4())
            # Create a document with metadata including the name and ID
            doc = Document(page_content=text, metadata={'name': name, 'id': person_id})
            documents.append(doc)
        
        return Chroma.from_documents(documents=documents, embedding=self.embeddings, persist_directory=self.vector_store_dir)

    def run(self, names):
        """Executes the process of loading biographies and creating a document store."""
        texts = self.load_biographies(names)
        vectorstore = self.create_document_store(texts)
        print("Vector store created and saved successfully.")

def retrieve_vector_store(vector_store_dir, embeddings):
    """Retrieves the vector store with the embedding function from the persisted directory."""
    return Chroma(persist_directory=vector_store_dir, embedding_function=embeddings)

In [3]:
# Example usage

# Test to see how Wikipedia interprets name input. Must test this before adding to the list of names to ensure the correct name is used. 
# Steve Jobs is actually recognized as Stev Jobs so be careful!
print(wikipedia.suggest("Jane Goodall")) # This returns Jane Goodal

names = ["Stev Jobs", "Jane Goodall", "Barack Obama"]
scraped_data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data/scraped')
vector_store_dir = os.path.join(os.path.dirname(os.getcwd()), 'data/vector_store')

biography_store = BiographyVectorStore(scraped_data_dir, vector_store_dir)
biography_store.run(names)

jane goodal


/home/mparoca/anaconda3/envs/pd_simulation/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/mparoca/anaconda3/envs/pd_simulation/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Successfully fetched biography for Stev Jobs.
Snippet: Steven Paul Jobs (February 24, 1955 – October 5, 2011) was an American businessman, inventor, and in...

Successfully fetched biography for Jane Goodal.
Snippet: Dame Jane Morris Goodall  (; born Valerie Jane Morris-Goodall; 3 April 1934), formerly Baroness Jane...

Successfully fetched biography for Barack Obama.
Snippet: Barack Hussein Obama II (born August 4, 1961) is an American politician who served as the 44th presi...

Vector store created and saved successfully.


In [4]:
# Test to see if the vector store can be retrieved
vectorstore = retrieve_vector_store(vector_store_dir, biography_store.embeddings)

In [6]:
# Test to see if vector store is working
print(vectorstore.similarity_search("chimpanzees"))


Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[Document(metadata={'id': 'cb2b0fd1-c95b-47ef-adcd-8a795eaef5be', 'name': 'Jane Goodal'}, page_content='Dame Jane Morris Goodall  (; born Valerie Jane Morris-Goodall; 3 April 1934), formerly Baroness Jane van Lawick-Goodall, is an English zoologist, primatologist and anthropologist. She is considered the world\'s foremost expert on chimpanzees, after 60 years\' studying the social and family interactions of wild chimpanzees. Goodall first went to Gombe Stream National Park in Tanzania to observe its chimpanzees in 1960.\nShe is the founder of the Jane Goodall Institute and the Roots & Shoots programme, and she has worked extensively on conservation and animal welfare issues. As of 2022, she is on the board of the Nonhuman Rights Project. In April 2002, she was named a United Nations Messenger of Peace. Goodall is an honorary member of the World Future Council.\n\n\n== Early life ==\nValerie Jane Morris-Goodall was born in April 1934 in Hampstead, London, to businessman Mortimer Herbert